In [11]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
import random
import itertools

In [ ]:
class City:
    def __init__(self,no_of_loc,locations=None):
    #here locations is a dictionary,{ nodeid: (x, y)} for specific locations
        self.G = nx.Graph()
        self.pos = {}
        self.num_nodes = no_of_loc
        if locations:
            self.pos = {k: np.array(v) for k, v in locations.items()}
            for i, coord in self.pos.items():
                self.G.add_node(i, pos=coord)
        else:
            self.pos = {i: np.array([random.uniform(0, 30), random.uniform(0, 30)]) for i in range(0, no_of_loc)}
            for i in range(0, no_of_loc):
                self.G.add_node(i, pos=self.pos[i])
        self._build_roads()
        self.edges = list(self.G.edges())
        self.edge_map = {tuple(sorted(e)): i for i, e in enumerate(self.edges)}
        self.num_edges = len(self.edges)
        self.true_alphas = np.random.uniform(-1.0, 1.0, self.num_edges)
        self.learned_alphas = np.zeros(self.num_edges)

        
    def _build_roads(self):
        nodes = list(self.G.nodes())
        threshold1 = 10.0 
        threshold2 = 25.0

        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)):
                u, v = nodes[i], nodes[j]
                d = float(np.linalg.norm(self.pos[u] - self.pos[v]))
                if d <= threshold1 or d>=threshold2:
                    self.G.add_edge(u, v, distance=d, weight=d)
                    
        if not nx.is_connected(self.G):
            components = list(nx.connected_components(G))
            print(components)
            for k in range(len(components)-1):
                u = random.choice(list(components[k]))
                v = random.choice(list(components[k+1]))
                d = float(np.linalg.norm(pos[a] - pos[b]))
                G.add_edge(u, v, distance=d)
                
    def get_distance_matrix(self,use_traffic=False):
        nodes = sorted(list(self.G.nodes()))
        n = len(nodes)
        matrix = np.full((n, n), np.inf)
        np.fill_diagonal(matrix, 0.0)
        for u, v, data in self.G.edges(data=True):
            if use_traffic:
                idx = self.edge_map.get(tuple(sorted((u, v))))
                alpha = self.learned_alphas[idx]
                cost = data['distance'] / (SPEED_BASE * np.exp(alpha))
                data['weights']=cost
            else:
                # Just Physical Distance
                cost = data['distance']
            matrix[u][v] = cost
            matrix[v][u] = cost
        return matrix, nodes
    def get_shortest_path_matrix(self, use_traffic=False):
        for u, v, data in self.G.edges(data=True):
            if use_traffic:
                idx = self.edge_map.get(tuple(sorted((u, v))))
                alpha = self.learned_alphas[idx]
                cost = data['distance'] / (SPEED_BASE * np.exp(alpha))
                data['weights']=cost
                str='weights'
            else:
                # Just Physical Distance
                cost = data['distance']
                str='distance'
        path_gen = nx.all_pairs_dijkstra_path_length(self.G, weight=str)
        n = self.num_nodes
        matrix = np.zeros((n, n))
        for src, targets in path_gen:
            for dst, val in targets.items():
                matrix[src][dst] = val
        
        return matrix
    def _get_edge_true_time(self, u, v,SPEED_BASE=40):
        idx = self.edge_map.get(tuple(sorted((u, v))))
        if idx is None: return 0
        return self.G[u][v]['distance'] / (SPEED_BASE * np.exp(self.true_alphas[idx]))
        
    def Tobs(self,route):
        total_time = 0.0
        for i in range(len(route) - 1):
            u, v = route[i], route[i+1]
            if self.G.has_edge(u, v):
                total_time += self._get_edge_true_time(u, v)
        return total_time
    def assign_orders_kmeans(self, agent_starts, order_nodes,iterations=20):
        distanceMatrix = self.get_shortest_path_matrix(use_traffic=False)
        # agent_starts contains nodes
        # self.pos[node] contains coordinates
        # initially centroids are coordinates of agent_starts
        centroids = agent_starts[:]
        k=len(agent_starts)
        clusters = [[] for _ in range(k)]
        for _ in range(iterations):
            clusters = [[] for _ in range(k)]
            
            for order in order_nodes:
                best_idx = -1
                min_dist = float("inf")
                
                for i in range(k):
                    depot_node = centroids[i]
                    dist = distanceMatrix[depot_node][order]

                    if dist < min_dist:
                        min_dist = dist
                        best_idx = i
                clusters[best_idx].append(order)
            new_centroids = []
            for i in range(k):
                cluster_orders = clusters[i]

                if not cluster_orders:
                    new_centroids.append(agent_starts[i])
                    continue

             
                best_node = None
                best_sum_dist = float("inf")

                for candidate in cluster_orders:
                    
                    total = sum(distanceMatrix[candidate][o] for o in cluster_orders)
                    
                    if total < best_sum_dist:
                        best_sum_dist = total
                        best_node = candidate

                new_centroids.append(best_node)

            centroids = new_centroids

        assignments = {}
        for i in range(k):
            assignments[agent_starts[i]] = clusters[i]
            
        return assignments
    def solve_tsp_dp(self, start_node, orders, use_traffic):
        full_dist_matrix = self.get_shortest_path_matrix(use_traffic)
        
        
        targets = [start_node] + orders
        n = len(targets)
        
       
        dist = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                dist[i][j] = full_dist_matrix[targets[i]][targets[j]]

        ALL_VISITED = (1 << n) - 1

        dp = [[math.inf] * n for _ in range(1 << n)]

        parent = [[-1] * n for _ in range(1 << n)]

        dp[1][0] = 0

        for mask in range(1 << n):
            if not (mask & 1):
                continue

            for curr in range(n):
                if not (mask & (1 << curr)):
                    continue
                
                if mask == 1 and curr == 0:
                    continue

                prev_mask = mask ^ (1 << curr)

                for prev in range(n):
                    if not (prev_mask & (1 << prev)):
                        continue
                    
                    new_cost = dp[prev_mask][prev] + dist[prev][curr]

                    if new_cost < dp[mask][curr]:
                        dp[mask][curr] = new_cost
                        parent[mask][curr] = prev

       
        min_cost = math.inf
        last_node_idx = -1

        for i in range(1, n):
            
            cost = dp[ALL_VISITED][i] 


            if cost < min_cost:
                min_cost = cost
                last_node_idx = i
        
        if last_node_idx == -1:
             return [start_node] + orders, float('inf')

        mask = ALL_VISITED
        curr = last_node_idx
        path_indices = []

        while curr != -1:
            path_indices.append(curr)
            prev = parent[mask][curr]
            mask ^= (1 << curr)
            curr = prev
        
        path_indices.reverse()
        
        optimal_route = [targets[i] for i in path_indices]

        return optimal_route, min_cost
        

    def route(self,start,orders,use_traffic=False):
        shortestpath= self.get_shortest_path_matrix(use_traffic)
        distance_matrix=self.get_distance_matrix(use_traffic)
        if not orders:
            return [start],0
        if len(orders) == 1:
            return [start,orders[0]],shortestpath[start][orders[0]]
        else:
            return self.solve_tsp_dp(start, orders,use_traffic)
   # def visualize_graph(self):
    #    plt.figure(figsize=(30, 30))
    #    nx.draw_networkx_nodes(self.G, self.pos, node_size=150, node_color="skyblue")
     #   nx.draw_networkx_edges(self.G, self.pos, width=1.5)
#
     #   nx.draw_networkx_labels(self.G, self.pos, font_size=12)
    #    edge_labels = {
    #        (u, v): f"{self.G[u][v]['distance']:.1f}"
    #        for u, v in self.G.edges()
     #   }
     #   nx.draw_networkx_edge_labels(self.G,self.pos, edge_labels=edge_labels, font_size=9)
     #   if title:
     #       plt.title(title)
     #   plt.axis("equal")
     #   plt.show()
     #   draw_graph(self.G, self.pos, title="Graph Visualization") ///
    def time_with_traffic(self,start,orders,route,tol=1e-4,max_iter=200,learning_rate=0.1,SPEED_BASE=40):
        t_obs = self.Tobs(route)
        #we calculate observed time using true alphas
        indices = []
        dists = []
        for i in range(len(route) - 1):
            u, v = route[i], route[i+1]
            if self.G.has_edge(u, v):
                indices.append(self.edge_map[tuple(sorted((u, v)))])
                dists.append(self.G[u][v]['distance'])
            else:
                pass
        if not indices :
            return
        
        indices = np.array(indices)
        dists = np.array(dists)

        for _ in range(max_iter):
            alphas = self.learned_alphas[indices]
            #bcz an edge has an alpha
            pred_edge_times = dists / (SPEED_BASE * np.exp(alphas))
            t_pred = np.sum(pred_edge_times)

            
            diff = t_pred - t_obs
            Loss = (t_pred - t_obs)**2
        
            
            if Loss < tol:
                break

            grads = 2 * diff * (-pred_edge_times) 
            for k in range(len(indices)):
                edge_idx = indices[k]
                g = grads[k]
                self.learned_alphas[edge_idx] = self.learned_alphas[edge_idx] - (learning_rate * g)

    def visualize_route(self, route, title="Delivery Route"):

        plt.figure(figsize=(10, 8))
        

        nx.draw(self.G, self.pos, 
                node_color='lightgray', node_size=50, 
                edge_color='lightgray', width=1, 
                with_labels=True, font_size=8)

        path_edges = []
        for i in range(len(route) - 1):
            u, v = route[i], route[i+1]
            if self.G.has_edge(u, v):
                path_edges.append((u, v))
            else:
                sub_path = nx.shortest_path(self.G, u, v, weight='distance')
                for k in range(len(sub_path)-1):
                    path_edges.append((sub_path[k], sub_path[k+1]))
        
        nx.draw_networkx_edges(self.G, self.pos, edgelist=path_edges, 
                               edge_color='blue', width=3)
        
        nx.draw_networkx_nodes(self.G, self.pos, nodelist=[route[0]], node_color='green', node_size=150, label="Start")
        nx.draw_networkx_nodes(self.G, self.pos, nodelist=[route[-1]], node_color='red', node_size=150, label="End")
        
        plt.title(f"{title}")
        plt.legend()
        plt.show()
        
    

In [ ]:
if __name__ == "__main__":
    NUM_NODES = 80
    NUM_AGENTS = 8
    NUM_ORDERS = 50
    No_of_times=100
    city = City(NUM_NODES)
    # print(city.G.nodes())
    for _ in range(No_of_times):
        all_nodes=list(city.G.nodes())
        agent_starts = random.sample(all_nodes, NUM_AGENTS)
        # print("agents")
        # print(agent_starts)
        remaining_nodes= [n for n in all_nodes if n not in agent_starts]
        all_orders = random.sample(remaining_nodes, NUM_ORDERS)
        assignments = city.assign_orders_kmeans(agent_starts, all_orders)
        for agent_id, assigned_orders in assignments.items():
            if not assigned_orders:
                continue
            route,cost = city.solve_tsp_dp(agent_id, assigned_orders,False)
            # print(f"Agent {agent_id} (Orders: {len(assigned_orders)}): Route Len {len(route)}, route: {route}, cost={cost}")
            city.time_with_traffic(agent_id,assigned_orders,route)

        # print("done")
    print(city.true_alphas)
    print(city.learned_alphas)
    print(route)
    #city.visualize_graph
        # print("orders")
        # print(all_orders)
        # print("assignments")
        # print(assignments)

TypeError: cannot unpack non-iterable NoneType object